

# 2023 华为云开源之夏项目结题报告

## 项目名称：基于昇思 MindSpore Quantum，实现用于求解分子体系本征态的非变分量子算法

项目编号：2349a0462  

项目导师：谢晴兴 

项目完成人：何润洪

### 1 课题背景：

可利用量子叠加和纠缠特性的量子算法在某些问题的求解上相比经典算法可具有指数级的加速效果。但容错量子计算的实现需要非常严苛的硬件条件支持，就当前的发展情况来看，量子硬件技术短期内将处于 NISQ 技术时代。基于 NISQ 设备特性设计量子算法解决实际难题是一个热门话题。

物质的化学性质与分子的能级相关（比如化学反应速率的预测需要依据分子基态能级），所以分子能级的精确计算是一项非常重要的任务。在经典计算模式下，分子的能级可用 FCI 算法进行精确求解，但该算法的计算成本随着分子尺寸增加呈指数上升趋势，所以不适用于大型分子。针对大型分子可采用近似算法进行求解，以适当牺牲精度为代价来降低计算成本，比如 CCSD、密度泛函等方法。这些方法可较为精确地求解波函数中只有一个电子结构占据主导的问题，但对于过渡金属等具有强关联系统的模拟则表现不佳。

第一个可用于求解哈密顿量本征值的量子算法是 QPE 算法。相比经典算法，QPE 可以提供指数级加速效果。但 QPE 所需的量子硬件条件，比如纠错机制以及长的相干时间，现阶段的 NISQ 设备尚难以满足。

最近，采用量子-经典混合架构的 VQE 算法受到了广泛关注。在 VQE 中，二次量子化后的哈密顿量可以通过 JW 变换或者 BK 变换映射为泡利算符串，从而将试探态对哈密顿量的期望值作为本征值。VQE 实际运行时，由量子芯片运行参数化量子线路（拟设线路）以生成试探态，再由经典计算机根据测量结果计算损失函数及梯度，从而更新拟设线路中的参数以获取更佳的结果。相比于 QPE 算法，VQE 算法的线路深度更浅，可以在 NISQ 时期实现。原始版本的 VQE 只能计算基态能级，后来也发展了很多新版本可以计算激发能级，比如子空间搜索变分算法等。虽然 VQE 对相干噪声具有一定的鲁棒性，不需要进行专门的纠错，但也存在两个缺点。首先是需要设计拟设线路，而如何合理地构建高效的拟设线路还是一个 Open 的话题。常用的构建方式可大致分为两类：基于理论和启发式。基于理论的方法有 UCCSD 等。而基于启发式的有 HEA 等，但这类方法容易受到贫瘠高原现象的困扰，很难有效对线路参数进行优化，而且也不能保证本征态就处于拟设线路的表达空间中。另外，即使构建出了有效的拟设线路，由于优化的非凸性，VQE 也不容易找到全局最优解。

也有基于非幺正操作的新方法被提出，比如虚实演化 ITE 算法。ITE 可以求解基态能级，但计算成本很高，随着分子轨道数呈指数级增加。为降低线路深度，ITE 算法也衍生出了诸多变体，如 VITE、QITE 以及 PITE。在 PITE 中，非幺正操作由对部分量子比特的测量引入。而由于整个 PITE 算法的完成有赖于每个子步骤的成功执行，所以最终求解成功的概率随子步骤的数量指数下降。

### 2 项目目标及拟采用的方案：

本项目的核心内容是实现一种新型的非变分量子算法——Power of Sine Hamiltonian Operator（PSHO）。该算法可用于求解量子系统的基态和激发态能级。相比于当前主流的变分类量子算法，PSHO 算法既不需要设计拟设线路，也不需要进行复杂的参数优化，因此具有巨大的应用潜力。

本项目基于 MindSpore Quantum 量子计算框架实现 PSHO 算法，探索该算法在求解分子体系本征态的性能，并与其他变分类量子算法从线路成本、采样成本、结果精度、噪声鲁棒性、算法通用性等方面进行比较分析，从而判断各种方法的优势与不足。

#### PSHO 算法

在 PSHO 算法中，随着执行次数 $n$ 的增加，初态 $|\varphi_{0}\rangle$ 会收敛到其按本征态展开的系数中 $|\sin(E_{i}\tau)|$ 最大值所对应能级 $E_{i}$ 的本征态，之后，通过测量，即可得到能级值 $E_{i}$。通过调节参数 $\tau$ 的值，可完成对基态和激发态能级的求解。

任意一个初态 $|\varphi_{0}\rangle$ 可以展开成给定哈密顿量 $\hat{H}$ 各本征态的线性组合形式

$$
|\varphi_{0}\rangle=\sum_{i}c_{i}|\Psi_{i}\rangle\text{，}\tag{1}
$$
其中， $|\Psi_{i}\rangle$​ 为哈密顿量的第 $i$ 个本征态，$c_{i}$ 为该本征态与 $|\varphi_{0}\rangle$ 之间的交叠系数。可定义
$$
|\varphi_{n}\rangle=\sin^{n}(\hat{H}\tau)|\varphi_{0}\rangle=\sum_{i}c_{i}\sin^{n}(E_{i}\tau)|\Psi_{i}\rangle\text{，}\tag{2}
$$
其中，$E_{i}$ 为 $|\Psi_{i}\rangle$ 的能量（本征能量）。 可以知道，随着执行次数 $n$ 增加，$|\varphi_{n}\rangle$ 会逐渐收敛到具有最大 $|\sin(\hat{H}\tau)|$ 值所对应的本征态上，且 $|\sin(\hat{H}\tau)|$ 值越大，收敛速度越快。而通过恰当选取 $\tau$ 即可挑选出待收敛的本征态，从而得到对应的能级。比如，要获得基态的话，可以通过下述两种方法：

### 直接法：

由于
$$
\lim_{n\rightarrow\infty}|\varphi_{n}\rangle=\lim_{n\rightarrow\infty}\sum_{i}c_{i}\sin^{n}(E_{i}\tau)|\Psi_{i}\rangle\thickapprox c_{0}\sin^{n}(E_{0}\tau)|\Psi_{0}\rangle\text{，}\tag{3}
$$

$$
\lim_{n\rightarrow\infty}E(|\varphi_{n}\rangle)=\lim_{n\rightarrow\infty}\frac{\langle\varphi_{n}|\hat{H}|\varphi_{n}\rangle}{\langle\varphi_{n}|\varphi_{n}\rangle}=E_{0}\text{。}\tag{4}
$$

如公式 (4)，通过在量子计算机上制备 $|\varphi_{n}\rangle$ 态，之后通过采样测量即可估计哈密顿量的基态能。由于 $\sin(\hat{H}\tau)$ 是非幺正算符，但一般的量子线路只能执行幺正的量子门。一种可行的办法是将非幺正的操作嵌入到一个更大的幺正操作中。虽然 $\sin(\hat{H}\tau)$ 是非幺正操作，但 $\sin^{2}(\hat{H}\tau)+\cos^{2}(\hat{H}\tau)=I$ 是幺正操作。于是这个“更大的幺正操作”可构造为
$$
\sum=\left(\begin{array}{cc}
\sin(\hat{H}\tau), & \cos(\hat{H}\tau)\\
\cos(\hat{H}\tau), & -\sin(\hat{H}\tau)
\end{array}\right)\text{。}\tag{5}
$$
考虑 $\sin(\hat{H}\tau)=\frac{i}{2}(\mathrm{e}^{-i\hat{H}\tau}-\mathrm{e}^{i\hat{H}\tau})$，$\cos(\hat{H}\tau)=\frac{1}{2}(\mathrm{e}^{-i\hat{H}\tau}+\mathrm{e}^{i\hat{H}\tau})$，操作 $\sum$ 所对应的量子线路如 图 1(a) 所示

![direct_circ.png](./src/direct_circ.png)

$\qquad\qquad\qquad\qquad\qquad$$图 ~1.\ \text{PSHO} 直接法计算量子体系能级的量子线路。$

在辅助比特初始状态为 $|0\rangle$ 的情况下，量子线路输出态为 
$$
|T_{3}\rangle=\left(\begin{array}{c}
\sin(\hat{H}\tau)|\varphi_{0}\rangle\\
\cos(\hat{H}\tau)|\varphi_{0}\rangle
\end{array}\right)。\tag{6}
$$
之后，对辅助比特进行测量，如果测量结果为 0，则寄存比特的状态会坍缩到 $\sin(\hat{H}\tau)|\varphi_{0}\rangle$。类推可知，实现操作 $\sin^{n}(\hat{H}\tau)$ 的量子线路可搭建为图 1(b)。而寄存器的初始态 $|\varphi_{0}\rangle$ 可通过一系列 X 门制备为 HF 态，或者采用 VQE 算法制备到近似基态以减小 PSHO 算法的执行次数 n。

从上面分析可知，要最终获得正确的 $|\varphi_{n}\rangle$ 态，需要对辅助比特的连续 $n$ 次测量结果都为 0，所以该方法最终成功制备 $|\varphi_{n}\rangle$ 态的概率随 $n$ 的增加呈指数降低，导致其在实验上不可行。所以本项目拟采用下面的间接法来实现 PSHO 对哈密顿量本征能的求解。

### 间接法：

$|\varphi_{n}\rangle$ 的正规化能量为

$$
E(|\varphi_{n}\rangle)=\frac{\langle\varphi_{n}|\hat{H}|\varphi_{n}\rangle}{\langle\varphi_{n}|\varphi_{n}\rangle}=\frac{\langle\varphi_{0}|\hat{H}\sin^{2n}(\hat{H}\tau)|\varphi_{0}\rangle}{\langle\varphi_{0}|\sin^{2n}(\hat{H}\tau)|\varphi_{0}\rangle}\text{。}\tag{7}
$$
代入 $\sin(\hat{H}\tau)=\frac{i}{2}(\mathrm{e}^{-i\hat{H}\tau}-\mathrm{e}^{i\hat{H}\tau})$，可得

$$
\begin{aligned}E(|\varphi_{n}\rangle) & =\frac{\langle\varphi_{0}|(\frac{i}{2})^{2n}\cdot\hat{H}\cdot(\mathrm{e}^{-i\hat{H}\tau}-\mathrm{e}^{i\hat{H}\tau})^{2n}|\varphi\rangle}{\langle\varphi_{0}|(\frac{i}{2})^{2n}\cdot(\mathrm{e}^{-i\hat{H}\tau}-\mathrm{e}^{i\hat{H}\tau})^{2n}|\varphi\rangle}\\
 & =\frac{\sum_{k=0}^{2n}(-1)^{2n-k}\cdot C_{2n}^{k}\cdot\langle\varphi_{0}|\hat{H}\mathrm{e}^{i\hat{H}\tau\cdot(2n-2k)}|\varphi_{0}\rangle}{\sum_{k=0}^{2n}(-1)^{2n-k}\cdot C_{2n}^{k}\cdot\langle\varphi_{0}|\mathrm{e}^{i\hat{H}\tau\cdot(2n-2k)}|\varphi_{0}\rangle}\text{。}
\end{aligned}\tag{8}
$$
为简明起见，我们将公式 (8) 的分子记为 $A_{n}$，而将分母记为 $B_{n}$。我们尝试在量子计算机上分别计算 $A_{n}$ 和 $B_{n}$。这样，再经过经典计算机简单处理就可以得到能量 $E(|\varphi_{n}\rangle)$。

考虑到 $C_{2n}^{k}=C_{2n}^{2n-k}$，公式 (8) 中分母的第 $k$ 项和第 $2n-k$ 项的系数是相同的，可以一起进行处理。于是有
$$
\begin{aligned} & (-1)^{2n-k}\cdot C_{2n}^{k}\cdot\langle\varphi_{0}|\mathrm{e}^{i\hat{H}\tau\cdot(2n-2k)}|\varphi_{0}\rangle+(-1)^{2n-(2n-k)}\cdot C_{2n}^{2n-k}\cdot\langle\varphi_{0}|\mathrm{e}^{i\hat{H}\tau\cdot(2n-2(2n-k))}|\varphi_{0}\rangle\\
 & =(-1)^{k}\cdot C_{2n}^{k}\cdot\langle\varphi_{0}|\mathrm{e}^{i\hat{H}\tau\cdot(2n-2k)}+\mathrm{e}^{i\hat{H}\tau\cdot(2n-2n)}|\varphi_{0}\rangle\\
 & =(-1)^{k}\cdot C_{2n}^{k}\cdot\langle\varphi_{0}|\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle\text{。}
\end{aligned}\tag{9}
$$
代入公式 (8) 的分母 $B_{n}$ 中，我们有
$$
B_{n}=\sum_{k=0}^{n-1}(-1)^{k}\cdot2C_{2n}^{k}\cdot\langle\varphi_{0}|\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle+C_{2n}^{n}\cdot(-1)^{n}。\tag{10}
$$

$$
A_{n}=\sum_{k=0}^{n-1}(-1)^{k}\cdot2C_{2n}^{k}\cdot\langle\varphi_{0}|\hat{H}\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle+C_{2n}^{n}\cdot(-1)^{n}+C_{2n}^{n}\cdot(-1)^{n}\langle\varphi_{0}|\hat{H}|\varphi_{0}\rangle\text{。}\tag{11}
$$

于是，能级求解的核心问题就转化为了在量子计算机上计算 $\langle\varphi_{0}|\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle$ 和 $\langle\varphi_{0}|\hat{H}\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle$ 这两项。

利用图 1(a) 所示的用于完成 $\sum$ 操作的量子线路，并且设演化时间为 $\tau(n-k)$，则输出态 $|T_{3}\rangle$ 为

$$
|T_{3}\rangle=\left(\begin{array}{c}
\sin[\hat{H}\tau(n-k)]|\varphi_{0}\rangle\\
\cos[\hat{H}\tau(n-k)]|\varphi_{0}\rangle
\end{array}\right)\text{。}\tag{12}
$$
对于量子态 $|T_{3}\rangle$，可观测量 $-\hat{Z}\otimes\hat{I}$ 和 $-\hat{Z}\otimes\hat{H}$ 的期望值分别为

$$
\begin{aligned}\langle T_{3}|-\hat{Z}\otimes\hat{I}|T_{3}\rangle & =\langle\varphi_{0}|\cos^{2}[\hat{H}\tau(n-k)]|\varphi_{0}\rangle-\langle\varphi_{0}|\sin^{2}[\hat{H}\tau(n-k)]|\varphi_{0}\rangle\\
 & =\langle\varphi_{0}|\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle\text{，}
\end{aligned}\tag{13}
$$
和

$$
\begin{aligned}\langle T_{3}|-\hat{Z}\otimes\hat{H}|T_{3}\rangle & =\langle\varphi_{0}|\hat{H}\cdot[\hat{H}\tau(n-k)]|\varphi_{0}\rangle-\langle\varphi_{0}|\hat{H}\cdot\sin^{2}[\hat{H}\tau(n-k)]|\varphi_{0}\rangle\\
 & =\langle\varphi_{0}|\hat{H}\cdot\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle\text{。}
\end{aligned}\tag{14}
$$
它们分别和公式 (8) 中 $A_{n}$ 和 $B_{n}$ 的第 $k$ 项相同。

于是，$|\varphi_{n}\rangle$ 的正规化能量就可以通过如图 2 所示的方案求得。

![indirect_circ.png](./src/indirect_circ.png)

$\qquad\qquad\qquad\qquad\qquad$$图~2$：$用于求解$ $\langle\varphi_{0}|\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle$ $和$ $\langle\varphi_{0}|\hat{H}\cdot\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle$ 

$\qquad\qquad\qquad\qquad\qquad$$项的量子线路图。两者需设定的可观测量分别为$ $-\hat{Z}\otimes\hat{H}$ $和$ $-\hat{Z}\otimes\hat{I}$。

将代入图 2 中的 $k$ 值从 0 增至 $n-1$，并获得 $-\hat{Z}\otimes\hat{H}$ 和 -$\hat{Z}\otimes\hat{I}$ 的期望值，即可得到公式 (10) 和公式 (11) 中的 $\langle\varphi_{0}|\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle$ 项和 $\langle\varphi_{0}|\hat{H}\cdot\cos[\hat{H}\tau\cdot2(n-k)]|\varphi_{0}\rangle$ 项。而公式 (11) 中最后一项中的 $\langle\varphi_{0}|\hat{H}|\varphi_{0}\rangle$ 则可以直接通过对初态的测量获得。如此，将上述量子计算机中得到的结果在经典计算机中进行简单操作，即可得到最终所求量子系统的基态。而通过调节参数 $\tau$ 即可依次求得其他激发态的能级。

相比于之前介绍的直接法，间接法的采样次数不会随着执行次数 $n$ 指数增加。虽然间接法也需要进行大量采样，但采样次数仅与映射哈密顿量的泡利算符数相关，而不是执行次数 $n$。因此，间接法比直接法更容易在实践中应用。

### 数据结果

本项目主要基于间接法实现 PSHO 算法并进行深入分析。主要程序和结果在 `directed_method.md` 文件中进行了展现。（另外，本项目也对直接法进行了复现。主要程序和结果在 `indirected_method.md` 文件中进行了展现。）相关可执行程序文件为：`psho_direct.py`，`psho_indirect.py`，`psho_noise.py`。

图 3 展示了采用 PSHO 间接法求解 H2 （键长 0.74）分子基态能量误差随着幂数 $n$ 增加的变化情况。

![error_indirect.png](./src/error_indirect.png)

$\qquad\qquad\qquad\qquad\qquad$ $图$ 3：PSHO $间接法计算基态能量的误差随幂数的变化关系。$

根据图 3 可知，随着幂数 $n$ 的增加，PSHO 间接法得到的基态能量越来越接近正确值。

图  4 对采用 PSHO 间接法求解 H2 分子基态能量在幂数 $n=10$ 时所用量子门数与 UCCSD 算法所用门数进行了对比。

![num_gate.png](./src/num_gate.png)

$\qquad\qquad\qquad\qquad\qquad$	$图$ 4：UCCSD $算法和不同幂数的$ PSHO $间接法计算$ H2 $基态能量所用量子门数量。$

根据图 4 可知，PSHO 间接法所用量子门数多于 UCCSD 算法。

图 5 展示了采用 PSHO 间接法求解 H2 （键长 0.74）分子基态能量误差在幂数 $n=10$ 时随噪声幅值的变化情况。其中振幅衰减信道的噪声门概率性地施加在了每个量子门后。

![noise_indirect.png](./src/noise_indirect.png)

$\qquad\qquad\qquad\qquad\qquad$ $图$ 5：PSHO $间接法计算基态能量的误差随噪声强度的变化关系。$

根据图 5 可知，随着噪声幅值的增加，PSHO 间接法得到的基态能量越来越偏离正确值。


![error_direct.png](./src/error_direct.png)
                $\qquad\qquad\qquad\qquad\qquad$ $图$ 6：PSHO $直接法计算基态能量的误差随辅助比特测量结果连续为 0 的次数的变化关系。$

图 6 展示了采用 PSHO 直接法求解 H2 （键长 0.74）分子基态能量误差随辅助比特测量结果 $n$ 次连续为 0  的变化情况。可见，随着 $n$ 增加，误差越来越小。









